Число ссылок на элемент

In [8]:
import sys
a = []
b = a
sys.getrefcount(a)

3

In [9]:
import sys
a = 6
b = a
sys.getrefcount(a)

347

In [10]:
import sys
sys.getrefcount(a)

346

In [1]:
a = 5
b =10 -5
c = 13-8
id(a)==id(b)==id(c)

True

Проблема заключалась в том, что эта переменная счетчика ссылок нуждалась в защите от условий гонки, когда два потока одновременно увеличивают или уменьшают ее значение. Если это произойдет, это может привести либо к утечке памяти, которая никогда не будет освобождена, либо, что еще хуже, к неправильному освобождению памяти, когда ссылка на этот объект все еще существует. Это может привести к сбоям или другим «странным» ошибкам в ваших программах на Python.

Эту переменную счетчика ссылок можно защитить, добавив блокировки ко всем структурам данных, которые совместно используются потоками, чтобы они не изменялись несогласованно.

Но добавление блокировки к каждому объекту или группе объектов означает, что будет существовать несколько блокировок, что может вызвать другую проблему — взаимные блокировки (взаимные блокировки могут возникнуть только при наличии более одной блокировки). Другим побочным эффектом будет снижение производительности, вызванное повторным получением и освобождением блокировок.

GIL — это единая блокировка самого интерпретатора, которая добавляет правило, согласно которому выполнение любого байт-кода Python требует получения блокировки интерпретатора. Это предотвращает взаимоблокировки (поскольку существует только одна блокировка) и не приводит к значительным потерям производительности. Но это эффективно делает любую программу Python, привязанную к процессору, однопоточной.

GIL, хотя и используется интерпретаторами для других языков, таких как Ruby, не является единственным решением этой проблемы. Некоторые языки избегают требования GIL для потокобезопасного управления памятью, используя подходы, отличные от подсчета ссылок, такие как сборка мусора.

С другой стороны, это означает, что эти языки часто должны компенсировать потерю однопоточного преимущества производительности GIL, добавляя другие функции повышения производительности, такие как JIT-компиляторы.

## Почему GIL был выбран в качестве решения?
Итак, почему подход, который кажется таким мешающим, используется в Python? Было ли это плохим решением разработчиков Python?

Что ж, по словам Ларри Гастингса , дизайнерское решение GIL — одна из вещей, которые сделали Python таким популярным, как сегодня.

Python существует с тех пор, когда операционные системы не имели концепции потоков. Python был разработан, чтобы быть простым в использовании, чтобы ускорить разработку, и все больше и больше разработчиков начали его использовать.

Было написано множество расширений для существующих библиотек C, функции которых были необходимы в Python. Чтобы предотвратить несогласованные изменения, эти расширения C требовали многопоточного управления памятью, которое обеспечивал GIL.

GIL прост в реализации и легко добавляется в Python. Это обеспечивает повышение производительности однопоточных программ, поскольку необходимо управлять только одной блокировкой.

Библиотеки C, которые не были потокобезопасными, стало легче интегрировать. И эти расширения C стали одной из причин, по которой Python был с готовностью принят различными сообществами.

Как видите, GIL был прагматичным решением сложной проблемы, с которой разработчики CPython столкнулись на заре существования Python.

---

In [19]:
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 2.3810031414031982


In [20]:
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1
#         if n < 3:
#             print("n < 3")

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))



start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()


end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 2.329993963241577


Как видите, обе версии занимают почти одинаковое количество времени. В многопоточной версии GIL предотвращал параллельное выполнение потоков, привязанных к ЦП.

GIL не оказывает большого влияния на производительность многопоточных программ, связанных с вводом-выводом, поскольку блокировка распределяется между потоками, пока они ожидают ввода-вывода.

Но программа, потоки которой полностью привязаны к процессору, например, программа, которая обрабатывает изображение по частям с использованием потоков, не только станет однопоточной из-за блокировки, но также увидит увеличение времени выполнения, как показано в приведенном выше примере. , по сравнению со сценарием, в котором он был написан полностью однопоточным.

Это увеличение является результатом накладных расходов на получение и освобождение, добавленных блокировкой.

## Почему GIL еще не удален?
Разработчики Python получают много жалоб по этому поводу, но такой популярный язык, как Python, не может внести столь значительные изменения, как удаление GIL, без проблем с обратной несовместимостью.

Очевидно, что GIL можно удалить, и это неоднократно делалось разработчиками и исследователями в прошлом, но все эти попытки нарушили существующие расширения C, которые сильно зависят от решения, предоставляемого GIL.

Конечно, есть и другие решения проблемы, которую решает GIL, но некоторые из них снижают производительность однопоточных и многопоточных программ, связанных с вводом-выводом, а некоторые просто слишком сложны. В конце концов, вы бы не хотели, чтобы ваши существующие программы на Python работали медленнее после выхода новой версии, верно?

Создатель и BDFL Python Гвидо ван Россум дал ответ сообществу в сентябре 2007 года в своей статье «Удалить GIL непросто» :
```
«Я бы приветствовал набор исправлений в Py3k только в том случае, если производительность для однопоточной программы (и для многопоточной, но связанной с вводом-выводом программы) не снизится »
```

И это условие не было выполнено ни одной из предпринятых с тех пор попыток.

In [ ]:
from multiprocessing import Pool
import time

COUNT = 50000000
def countdown(n):
    while n>0:
        n -= 1

if __name__ == '__main__':
    pool = Pool(processes=2)
    start = time.time()
    r1 = pool.apply_async(countdown, [COUNT//2])
    r2 = pool.apply_async(countdown, [COUNT//2])
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)

---

In [1]:
import time
import requests
import concurrent.futures


def get_wiki_page_existence(wiki_page_url, timeout=10):
    response = requests.get(url=wiki_page_url, timeout=timeout)

    page_status = "unknown"
    if response.status_code == 200:
        page_status = "exists"
    elif response.status_code == 404:
        page_status = "does not exist"

    return wiki_page_url + " - " + page_status

wiki_page_urls = ["https://en.wikipedia.org/wiki/" + str(i) for i in range(50)]

print("Running without threads:")
without_threads_start = time.time()
for url in wiki_page_urls:
    print(get_wiki_page_existence(wiki_page_url=url))
print("Without threads time:", time.time() - without_threads_start)

Running without threads:
https://en.wikipedia.org/wiki/0 - exists
https://en.wikipedia.org/wiki/1 - exists
https://en.wikipedia.org/wiki/2 - exists
https://en.wikipedia.org/wiki/3 - exists
https://en.wikipedia.org/wiki/4 - exists
https://en.wikipedia.org/wiki/5 - exists
https://en.wikipedia.org/wiki/6 - exists
https://en.wikipedia.org/wiki/7 - exists
https://en.wikipedia.org/wiki/8 - exists
https://en.wikipedia.org/wiki/9 - exists
https://en.wikipedia.org/wiki/10 - exists
https://en.wikipedia.org/wiki/11 - exists
https://en.wikipedia.org/wiki/12 - exists
https://en.wikipedia.org/wiki/13 - exists
https://en.wikipedia.org/wiki/14 - exists
https://en.wikipedia.org/wiki/15 - exists
https://en.wikipedia.org/wiki/16 - exists
https://en.wikipedia.org/wiki/17 - exists
https://en.wikipedia.org/wiki/18 - exists
https://en.wikipedia.org/wiki/19 - exists
https://en.wikipedia.org/wiki/20 - exists
https://en.wikipedia.org/wiki/21 - exists
https://en.wikipedia.org/wiki/22 - exists
https://en.wikipedi

In [1]:
import time
import requests
import concurrent.futures


def get_wiki_page_existence(wiki_page_url, timeout=10):
    response = requests.get(url=wiki_page_url, timeout=timeout)

    page_status = "unknown"
    if response.status_code == 200:
        page_status = "exists"
    elif response.status_code == 404:
        page_status = "does not exist"

    return wiki_page_url + " - " + page_status
wiki_page_urls = ["https://en.wikipedia.org/wiki/" + str(i) for i in range(500)]

print("Running threaded:")
threaded_start = time.time()
with concurrent.futures.ThreadPoolExecutor(12) as executor:
    futures = []
    for url in wiki_page_urls:
        futures.append(executor.submit(get_wiki_page_existence, wiki_page_url=url))
    #print(futures)
    for future in concurrent.futures.as_completed(futures):
        print(future.result())
print("Threaded time:", time.time() - threaded_start)

Running threaded:
https://en.wikipedia.org/wiki/2 - exists
https://en.wikipedia.org/wiki/4 - exists
https://en.wikipedia.org/wiki/8 - exists
https://en.wikipedia.org/wiki/0 - exists
https://en.wikipedia.org/wiki/1 - exists
https://en.wikipedia.org/wiki/3 - exists
https://en.wikipedia.org/wiki/13 - exists
https://en.wikipedia.org/wiki/15 - exists
https://en.wikipedia.org/wiki/14 - exists
https://en.wikipedia.org/wiki/16 - exists
https://en.wikipedia.org/wiki/19 - exists
https://en.wikipedia.org/wiki/6 - exists
https://en.wikipedia.org/wiki/17 - exists
https://en.wikipedia.org/wiki/12 - exists
https://en.wikipedia.org/wiki/21 - exists
https://en.wikipedia.org/wiki/11 - exists
https://en.wikipedia.org/wiki/10 - exists
https://en.wikipedia.org/wiki/7 - exists
https://en.wikipedia.org/wiki/18 - exists
https://en.wikipedia.org/wiki/23 - exists
https://en.wikipedia.org/wiki/5 - exists
https://en.wikipedia.org/wiki/9 - exists
https://en.wikipedia.org/wiki/22 - exists
https://en.wikipedia.org/w

https://en.wikipedia.org/wiki/190 - exists
https://en.wikipedia.org/wiki/191 - exists
https://en.wikipedia.org/wiki/199 - exists
https://en.wikipedia.org/wiki/201 - exists
https://en.wikipedia.org/wiki/202 - exists
https://en.wikipedia.org/wiki/203 - exists
https://en.wikipedia.org/wiki/205 - exists
https://en.wikipedia.org/wiki/204 - exists
https://en.wikipedia.org/wiki/206 - exists
https://en.wikipedia.org/wiki/213 - exists
https://en.wikipedia.org/wiki/212 - exists
https://en.wikipedia.org/wiki/187 - exists
https://en.wikipedia.org/wiki/211 - exists
https://en.wikipedia.org/wiki/216 - exists
https://en.wikipedia.org/wiki/208 - exists
https://en.wikipedia.org/wiki/219 - exists
https://en.wikipedia.org/wiki/217 - exists
https://en.wikipedia.org/wiki/200 - exists
https://en.wikipedia.org/wiki/222 - exists
https://en.wikipedia.org/wiki/221 - exists
https://en.wikipedia.org/wiki/225 - exists
https://en.wikipedia.org/wiki/220 - exists
https://en.wikipedia.org/wiki/218 - exists
https://en.

https://en.wikipedia.org/wiki/396 - exists
https://en.wikipedia.org/wiki/398 - exists
https://en.wikipedia.org/wiki/387 - exists
https://en.wikipedia.org/wiki/402 - exists
https://en.wikipedia.org/wiki/403 - exists
https://en.wikipedia.org/wiki/406 - exists
https://en.wikipedia.org/wiki/390 - exists
https://en.wikipedia.org/wiki/405 - exists
https://en.wikipedia.org/wiki/400 - exists
https://en.wikipedia.org/wiki/409 - exists
https://en.wikipedia.org/wiki/408 - exists
https://en.wikipedia.org/wiki/399 - exists
https://en.wikipedia.org/wiki/410 - exists
https://en.wikipedia.org/wiki/413 - exists
https://en.wikipedia.org/wiki/407 - exists
https://en.wikipedia.org/wiki/414 - exists
https://en.wikipedia.org/wiki/395 - exists
https://en.wikipedia.org/wiki/416 - exists
https://en.wikipedia.org/wiki/404 - exists
https://en.wikipedia.org/wiki/419 - exists
https://en.wikipedia.org/wiki/417 - exists
https://en.wikipedia.org/wiki/412 - exists
https://en.wikipedia.org/wiki/411 - exists
https://en.